## Data quality check / cleaning / preparation 

Put code with comments. The comments should explain the code such that it can be easily understood. You may put text *(in a markdown cell)* before a large chunk of code to explain the overall purpose of the code, if it is not intuitive. **Put the name of the person / persons who contributed to each code chunk / set of code chunks.** An example is given below.

### Distribution of variables
*By Sylvia Sherwood*

In [ ]:
#...Plot for distribution of variables...#

### Data cleaning
*By Luna Xu*

In [ ]:
# Importing libraries
import pandas as pd
from shapely.geometry import Point
from shapely import wkt
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import geopandas as gpd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from collections import Counter

# Reading Data
project_data = pd.read_csv('My_CHI._My_Future._Programs_20241113.csv')
chi_nei=pd.read_csv('CommAreas_20241114.csv')

# Exclude Online Program
project_data['Geographic Cluster Name'] = project_data.apply(
    lambda row: 'online' if row['Meeting Type'] == 'online' and pd.isnull(row['Geographic Cluster Name']) else row['Geographic Cluster Name'],
    axis=1
)

# Comparing Geographic Cluster Names and neighborhoodNames in Community Boundaries
project_data_unique = project_data['Geographic Cluster Name'].unique()
chi_nei_unique = chi_nei['COMMUNITY'].unique()
matches = set(project_data_unique).intersection(chi_nei_unique)
unmatched_project_data = set(project_data_unique) - matches
unmatched_chi_nei = set(chi_nei_unique) - matches
print(f"Matches: {matches}")
print(f"Unmatched in project_data: {unmatched_project_data}")
print(f"Unmatched in chi_nei: {unmatched_chi_nei}")

# Extracting Programs with No Geographic Cluster Name or Unstandardized Name
unmatched_geocluster_list = list(unmatched_project_data)
project_withlatlong=project_data.loc[
    ((project_data['Geographic Cluster Name'].isnull()) | (project_data['Geographic Cluster Name'].isin(unmatched_geocluster_list))) & 
    (project_data['Latitude'].notnull()) & 
    (project_data['Longitude'].notnull()),
    ['Program ID','Latitude','Longitude','Geographic Cluster Name']
]

# Turning data into shapely format & Mapping 
project_withlatlong['point_geom']=project_withlatlong.apply(
    lambda row: Point(row['Longitude'],row['Latitude']),axis=1
)
chi_nei['shapely_geom']=chi_nei['the_geom'].apply(wkt.loads)
def match_multiploygon(point,multipolygons):
    for muultipolygon in multipolygons:
        if muultipolygon.contains(point):
            return muultipolygon
    return None
project_withlatlong['shapely_geom']=project_withlatlong['point_geom'].apply(
    lambda point: match_multiploygon(point,chi_nei['shapely_geom'])
)
matched_program_neiname = pd.merge(project_withlatlong,chi_nei,how='left')

# Checking Unstandardized Name & the Neighborhood They Mapped to
print(matched_program_neiname.groupby('Geographic Cluster Name')['COMMUNITY'].unique())

# Result, imputed dataset
attempt1_result = matched_program_neiname.loc[matched_program_neiname['COMMUNITY'].notnull(),['Program ID','COMMUNITY']]
project_data = pd.merge(project_data,attempt1_result,on='Program ID',how='left')
project_data['Neighborhood'] = project_data.apply(
    lambda row: row['COMMUNITY'] if pd.notnull(row['COMMUNITY']) else row['Geographic Cluster Name'],
    axis=1
)
project_data.loc[project_data['Neighborhood'].isin(unmatched_geocluster_list),'Neighborhood']= None
project_data.loc[project_data['Geographic Cluster Name']=='online',['Neighborhood']]= 'online'
project_data=project_data.drop(columns=['COMMUNITY'])

### Data preparation
*By Luna Xu*


In [ ]:
######---------------Creating new variables----------------#########

#Creating three socioeconomic status bins
chi_ses = pd.read_csv('Census_Data_-_Selected_socioeconomic_indicators_in_Chicago__2008___2012.csv')
chi_ses['ses_bin'] = pd.cut(chi_ses['HARDSHIP INDEX'],3,labels=['High-SES','Mid-SES','Low-SES'])
chi_ses=chi_ses[chi_ses['COMMUNITY AREA NAME']!='CHICAGO']#drop chicago total measure

#Creating Start Year Variable from Start Date 
project_data['Start Date']=pd.to_datetime(project_data['Start Date'],format='%m/%d/%Y')
project_data['Start Year']=project_data['Start Date'].dt.year

## Exploratory data analysis

Put code with comments. The comments should explain the code such that it can be easily understood. You may put text *(in a markdown cell)* before a large chunk of code to explain the overall purpose of the code, if it is not intuitive. **Put the name of the person / persons who contributed to each code chunk / set of code chunks.**

### Analysis 1
*By \<Name of person doing the analysis>*

### Analysis 2
*By Luna Xu*

#### Program Distributions

In [ ]:
# merge census dataset with MCMF dataset
project_ses=pd.merge(project_data,chi_ses,left_on='Neighborhood',right_on='COMMUNITY AREA NAME',how='left')
# filter MCMF in-person programs with neighborhood info
project_hasnei_ses=project_ses.loc[project_ses['Neighborhood'].notnull() & (project_ses['Neighborhood']!='online')] 
# merge census dataset with neighborhood boundary dataset
chi_ses['COMMUNITY AREA NAME']=chi_ses['COMMUNITY AREA NAME'].astype(str).str.upper()
chi_nei_ses=pd.merge(chi_nei,chi_ses,left_on='COMMUNITY',right_on='COMMUNITY AREA NAME')
# create geopandas dataset
chi_nei_ses_gdf=gpd.GeoDataFrame(chi_nei_ses,geometry='shapely_geom')
# count number of distinct programs by neighborhood
program_counts = project_hasnei_ses.groupby('Neighborhood')['Program ID'].nunique().reset_index()
program_counts['Program Count']=program_counts['Program ID']
program_counts['Program Count']=program_counts['Program Count'].fillna(0)
# merge program count with neighborhood geographic & ses information
program_count_nei=pd.merge(program_counts,chi_nei_ses_gdf,left_on='Neighborhood',right_on='COMMUNITY')
# create geopandas dataset of the previous dataset
program_count_nei_gdf=gpd.GeoDataFrame(program_count_nei,geometry='shapely_geom')
# slice dataset by ses
lowses=program_count_nei_gdf.loc[program_count_nei_gdf['ses_bin']=='Low-SES',:]
midses=program_count_nei_gdf.loc[program_count_nei_gdf['ses_bin']=='Mid-SES',:]
highses=program_count_nei_gdf.loc[program_count_nei_gdf['ses_bin']=='High-SES',:]
#geospatial plotting
fig, axes = plt.subplots(1, 3, figsize=(18, 6), constrained_layout=True)
# Plot for low SES neighborhood
lowses.plot(ax=axes[0], column='Program Count', cmap='YlGnBu', 
            norm=LogNorm(vmin=program_count_nei_gdf['Program Count'].min(), 
                         vmax=program_count_nei_gdf['Program Count'].max()), 
            alpha=0.7)
for idx, row in lowses.iterrows():
    centroid = row['shapely_geom'].centroid
    axes[0].annotate(text=int(row['Program Count']),
                     xy=(centroid.x, centroid.y),
                     fontsize=7, ha='center', color='black', fontweight='bold')
axes[0].set_title('Low-SES Neighborhood')
# Plot for mid SES neighborhood
midses.plot(ax=axes[1], column='Program Count', cmap='YlGnBu', 
            norm=LogNorm(vmin=program_count_nei_gdf['Program Count'].min(), 
                         vmax=program_count_nei_gdf['Program Count'].max()), 
            alpha=0.7)
for idx, row in midses.iterrows():
    centroid = row['shapely_geom'].centroid
    axes[1].annotate(text=int(row['Program Count']),
                     xy=(centroid.x, centroid.y),
                     fontsize=7, ha='center', color='black', fontweight='bold')
axes[1].set_title('Mid-SES Neighborhood')
# Plot for high SES neighborhood
highses.plot(ax=axes[2], column='Program Count', cmap='YlGnBu', 
             norm=LogNorm(vmin=program_count_nei_gdf['Program Count'].min(), 
                          vmax=program_count_nei_gdf['Program Count'].max()), 
             alpha=0.7)
for idx, row in highses.iterrows():
    centroid = row['shapely_geom'].centroid
    axes[2].annotate(text=int(row['Program Count']),
                     xy=(centroid.x, centroid.y),
                     fontsize=7, ha='center', color='black', fontweight='bold')
axes[2].set_title('High-SES Neighborhood')
fig.suptitle('Program Distribution Based On Neighborhood SES', fontsize=16)
plt.show()

In [ ]:
# merge MCMF dataset with neighborhood ses
project_by_year_by_ses=pd.merge(project_data,chi_ses,left_on="Neighborhood",right_on="COMMUNITY AREA NAME")
# exclude 2025 data
project_by_year_by_ses=project_by_year_by_ses[project_by_year_by_ses['Start Year']!=2025]
# count distinct programs based on ses & year
year_nei_programcount=project_by_year_by_ses.groupby(['Start Year','ses_bin'])['Program ID'].nunique().reset_index()
year_nei_programcount=year_nei_programcount.rename(columns={'Program ID':'Program Count'})
pivotdf=year_nei_programcount.pivot(index='Start Year',columns='ses_bin',values='Program Count')
# create lineplot of program coun by neighborhood ses over time
plt.figure(figsize=(10,5))
for ses in pivotdf.columns:
    plt.plot(pivotdf.index,pivotdf[ses], label=ses)
    for year,count in zip(pivotdf.index,pivotdf[ses]):
        if year == 2020:
            offset = 500 * (pivotdf.columns.tolist().index(ses) + 1)
            plt.text(year, count + offset, str(count), fontsize=8, ha='center', va='top')
        else:
            plt.text(year, count, str(count), fontsize=8, ha='center', va='bottom')
plt.title("Program Count By Neighborhood SES Over Time")
plt.xlabel("Year")
plt.ylabel("Total Number Program")
plt.xticks(ticks=pivotdf.index)
plt.legend(title="Neighborhood SES")
plt.show()

In [ ]:
# count distinct programs by neighborhood and year
pivot_distro_data = (
    project_by_year_by_ses
    .groupby(['Start Year', 'Neighborhood'])['Program ID']
    .nunique()
    .reset_index()
    .rename(columns={'Program ID':'Program Count'})
)
pivot_distro_data = pivot_distro_data.merge(
    project_by_year_by_ses[['Start Year', 'Neighborhood', 'ses_bin']].drop_duplicates(),
    on=['Start Year', 'Neighborhood'],
    how='left'
)
# create side-by-side boxplot
sns.boxplot(pivot_distro_data,x='Start Year',y='Program Count',hue='ses_bin').set_title('A Comparsion of the Distribution of Programs among Neighborhood By SES')
# show the top 10 neighborhood with top program count
pivot_distro_data.sort_values(by='Program Count',ascending=False).head(10)

In [ ]:
# definition a function to create heatmap based on ses and year
def generate_heatmap(data, title, label, col_label, row_label, ax):
    heatmap_data = data.pivot(index='ses_bin', columns='Start Year', values='Offers').fillna(0)
    sns.heatmap(heatmap_data, annot=True, fmt=".0f", cmap="YlOrBr", cbar_kws={'label': label}, ax=ax)
    ax.set_title(title)
    ax.set_xlabel(col_label)
    ax.set_ylabel(row_label)
fig, axes = plt.subplots(1,3, figsize=(14.5, 3.5))
# Scholarship Heatmap
scholarship_offers_by_neises = (
    project_by_year_by_ses[project_by_year_by_ses['Scholarship Available'] == True]
    .groupby(['ses_bin', 'Start Year'])
    .agg(Offers=('Program ID', 'nunique'))
    .reset_index()
)
generate_heatmap(scholarship_offers_by_neises, "Scholarship Programs by Neighborhood-SES and Year", 
                 "Number of Scholarships Programs", "Year", "Neighborhood", axes[0])
# Free Food Heatmap
freefood_by_neises = (
    project_by_year_by_ses[project_by_year_by_ses['Has Free Food'] == True]
    .groupby(['ses_bin', 'Start Year'])
    .agg(Offers=('Program ID', 'nunique'))
    .reset_index()
)
generate_heatmap(freefood_by_neises, "Free Food Programs by Neighborhood-SES and Year", 
                 "Number of Free Food Programs", "Year", "Neighborhood", axes[1])
# Paid Programs Heatmap
paid_offers_by_neises = (
    project_by_year_by_ses[project_by_year_by_ses['Participants Paid'] == 'Paid, Type Unknown']
    .groupby(['ses_bin', 'Start Year'])
    .agg(Offers=('Program ID', 'nunique'))
    .reset_index()
)
# deal with no paid program 2020
fillna2020 = pd.DataFrame({
    'ses_bin': ['High-SES', 'Mid-SES', 'Low-SES'],
    'Start Year': [2020, 2020, 2020],
    'Offers': [0, 0, 0]
})
paid_offers_by_neises = pd.concat([paid_offers_by_neises, fillna2020], ignore_index=True)
ses_order = ['High-SES', 'Mid-SES', 'Low-SES']
paid_offers_by_neises['ses_bin'] = pd.Categorical(paid_offers_by_neises['ses_bin'], categories=ses_order, ordered=True)
paid_offers_by_neises=paid_offers_by_neises.sort_values(by=['ses_bin', 'Start Year']).reset_index(drop=True)
generate_heatmap(paid_offers_by_neises, "Paid Programs by Neighborhood-SES and Year", 
                 "Number of Paid Programs", "Year", "Neighborhood SES", axes[2])
plt.tight_layout()
plt.show()


#### Equity Feature: Scholarship Available

In [ ]:
# group program categories
project_by_year_by_ses['Category Group']=project_by_year_by_ses['Category Name'].map(
    {
       'Work + Career':'Career & Life Skills', 
       'Managing Money.':'Career & Life Skills', 
       'Reading & Writing.':'STEM & Writing',
       'Music & Art.':'Arts & Humanity', 
       'Sports + Wellness.':'Sports & Wellbeing', 
       'Science':'STEM & Writing', 
       'Food.':'Sports & Wellbeing',
       'Academic Support':'STEM & Writing', 
       'Digital Media.':'Arts & Humanity', 
       'Performance.':'Arts & Humanity', 
       'Healthcare':'Sports & Wellbeing',
       'Social Studies':'Arts & Humanity', 
       'Computers.':'STEM & Writing', 
       'Math':'STEM & Writing', 
       'Helping Your Community.':'Arts & Humanity',
       'Building & Fixing Things':'Career & Life Skills', 
       'Nature.':'Sports & Wellbeing', 
       'Teaching':'Career & Life Skills',
       'Customer/Human Service':'Career & Life Skills', 
       'Transportation':'Career & Life Skills', 
       'Law':'Career & Life Skills'
    }
)
# count distinct programs based on ses and category
ses_category_programcount = project_by_year_by_ses[project_by_year_by_ses['Scholarship Available']==True].loc[:,['Program ID','Category Group','ses_bin']].groupby(
    ['ses_bin','Category Group']
    ).agg(
        NumofProgram=('Program ID', 'nunique')
        ).reset_index().sort_values(by=['ses_bin', 'NumofProgram'], ascending=[True, False])
# side-by-side barplot of scholarship program category distribution by SES
sns.barplot(ses_category_programcount,x='ses_bin',y='NumofProgram',hue='Category Group').set_title('Scholarship Program Category Distribution by Neighborhood SES')
plt.legend(title='Program Category Group', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()


#### Equity Feature: Has Free Food

In [ ]:
# count freefood programs by neighborhood
freefood_by_nei = (
    project_by_year_by_ses[project_by_year_by_ses['Has Free Food'] == True]
    .groupby(['Neighborhood'])
    .agg(Offers=('Program ID', 'nunique'))
    .reset_index()
)
# merge the previous dataset to get hardship index for each neighborhood
freefood_by_nei=freefood_by_nei.merge(project_by_year_by_ses[['HARDSHIP INDEX','ses_bin','Neighborhood']].drop_duplicates(),on='Neighborhood')
# plot scatter with trendline
sns.regplot(freefood_by_nei,x='HARDSHIP INDEX',y='Offers')
plt.xlabel('Neighborhood Hardship Index')
plt.ylabel('Number of Free Food Programs')
plt.title('The Relationship Between Neighborhood Hardship Level and Number of Free Food Programs Offered')
# find the top and bottom neighborhood with free food programs
freefood_by_nei.sort_values(by='Offers',ascending=False).head(10)
freefood_by_nei.sort_values(by='Offers',ascending=False).tail(10)

#### Equity Feature: Participants Paid

In [ ]:
# get all paid programs
paid_programs=project_by_year_by_ses[project_by_year_by_ses['Participants Paid'] == 'Paid, Type Unknown'].loc[:,['Program ID','Description','Org Name']]
paid_programs=paid_programs.set_index('Program ID').drop_duplicates()
# download relevant nltk packages
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
stop_words = set(stopwords.words('english'))
# create a function that clean and slice descriptions by word
def description_cleaner(description):
    cleaned_description = re.sub(r'<.*?>', '', description)
    words = word_tokenize(cleaned_description.lower())
    return [word for word in words if word not in stop_words and word.isalpha()]
# create a column that store clean description (list of words)
paid_programs['Cleaned_Description']=paid_programs['Description'].apply(description_cleaner)
# get all words from all descriptions
all_words = [word for description in paid_programs['Cleaned_Description'] for word in description]
# count word frequency
word_counts = Counter(all_words)
# Get the most common words
common_words = word_counts.most_common(20) 
# plot a bar graph to show word frequency
word, frequency = zip(*common_words)
plt.figure(figsize=(6, 4))
plt.bar(word, frequency, color='skyblue')
plt.title('Word Frequencies in Paid Program Description', fontsize=16)
plt.xlabel('Words', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.xticks(rotation=45, ha='right', fontsize=10)  
plt.tight_layout() 
plt.show()
# create a function that slice description into sentences
def sentence_seperator(description,word):
    cleaned_description = re.sub(r'<.*?>', '', description)
    sentences = re.split(r'(?<=[.!?])', cleaned_description)
    return [sentence for sentence in sentences if word.lower() in sentence.lower()]
# find the sentences in descriptions that contain the word "paid"
paidword_program=project_by_year_by_ses[project_by_year_by_ses['Description'].str.contains('paid', case=False, na=False)]
paidword_program=paidword_program.loc[:,['Program ID','Program Name', 'Description','COMMUNITY AREA NAME','Scholarship Available','Participants Paid']]
paidword_program=paidword_program.set_index('Program ID').drop_duplicates()
not_labeled_paidword_program=paidword_program[(paidword_program['Participants Paid'] != 'Paid, Type Unknown') & (paidword_program['Scholarship Available'] == False)]
not_labeled_paidword_program.loc[:,['Matching Sentences']] = not_labeled_paidword_program['Description'].apply(lambda desc: sentence_seperator(desc,'paid'))
# display five samples of sentence that contains "paid" to inspect whether it actually reflects that the program pays participants
pd.set_option('display.max_colwidth', None)
not_labeled_paidword_program.loc[:,['Matching Sentences']].sample(5)
# find the sentences in descriptions that contain the word "stipend"
stipend_program=project_by_year_by_ses[project_by_year_by_ses['Description'].str.contains('stipend', case=False, na=False)]
stipend_program=stipend_program.loc[:,['Program ID','Program Name', 'Description','COMMUNITY AREA NAME','Scholarship Available','Participants Paid']]
stipend_program=stipend_program.set_index('Program ID').drop_duplicates()
not_labeled_stipend_program=stipend_program[(stipend_program['Participants Paid'] != 'Paid, Type Unknown') & (stipend_program['Scholarship Available'] == False)]
not_labeled_stipend_program.loc[:,['Matching Sentences']] = not_labeled_stipend_program['Description'].apply(lambda desc: sentence_seperator(desc, 'stipend'))
# display five samples of sentence that contains "stipend" to inspect whether it actually reflects that the program pays participants
not_labeled_stipend_program.loc[:,['Matching Sentences']].sample(5)
# create a column called stipend and impute it with true if has matching sentence 
project_by_year_by_ses.loc[
    project_by_year_by_ses['Program ID'].isin(not_labeled_stipend_program.index),
    'Stipend'
] = True
# count distinct paid programs based on ses and start year 
new_paid_offers_by_neises = (
    project_by_year_by_ses[(project_by_year_by_ses['Participants Paid'] == 'Paid, Type Unknown') | (project_by_year_by_ses['Stipend']==True)]
    .groupby(['ses_bin', 'Start Year'])
    .agg(Offers=('Program ID', 'nunique'))
    .reset_index()
)
new_paid_offers_by_neises = pd.concat([new_paid_offers_by_neises, fillna2020], ignore_index=True)
new_paid_offers_by_neises['ses_bin'] = pd.Categorical(new_paid_offers_by_neises['ses_bin'], categories=ses_order, ordered=True)
new_paid_offers_by_neises=new_paid_offers_by_neises.sort_values(by=['ses_bin', 'Start Year']).reset_index(drop=True)
# plot two heatmap to compare the result before and after imputing stipend programs
fig, axes = plt.subplots(1, 2, figsize=(12, 3))

generate_heatmap(paid_offers_by_neises, "Paid Programs by Neighborhood-SES and Year", 
                 "Number of Paid Programs", "Year", "Neighborhood SES", axes[0])

generate_heatmap(new_paid_offers_by_neises, "Stipend & Paid Programs by Neighborhood-SES and Year", 
                 "Number of Paid Programs", "Year", "Neighborhood SES", axes[1])

plt.tight_layout()
plt.show()

### Analysis 3
*By \<Name of person doing the analysis>*

### Analysis 4
*By \<Name of person doing the analysis>*

## Other sections

Put code with comments. The comments should explain the code such that it can be easily understood. You may put text *(in a markdown cell)* before a large chunk of code to explain the overall purpose of the code, if it is not intuitive. **Put the name of the person / persons who contributed to each code chunk / set of code chunks.**

Put each model in a section of its name and mention the name of the team-member tuning the model. Below is an example:

## Conclusions and Recommendations to stakeholder(s)

You may or may not have code to put in this section. Delete this section if it is irrelevant.